In [1]:
import yaml
with open("configs/hyperparameters.yaml") as f:
    config_dict =yaml.load(f, Loader=yaml.FullLoader)

In [2]:
from envs.photo_env import PhotoEnhancementEnv
env = PhotoEnhancementEnv(64)

Encoding testing data ...


  0%|          | 0/4 [00:00<?, ?it/s]

finished...
Encoding training data ...


  0%|          | 0/36 [00:00<?, ?it/s]

finished...


In [3]:
class Config(object):
    def __init__(self, dictionary):
        self.__dict__.update(dictionary)
sac_config = Config(config_dict)

In [4]:
import random
import numpy as np
import torch
SEED = sac_config.seed
DEVICE= 'CUDA'

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = sac_config.torch_deterministic

In [5]:
from torch.utils.tensorboard import SummaryWriter
import time
run_name = f"{sac_config.exp_name}__{sac_config.seed}__{int(time.time())}"
writer = SummaryWriter(f"runs/{run_name}")
writer.add_text(
    "hyperparameters",
    "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(sac_config).items()])),
)

In [6]:
from sac.sac_algorithm import SAC
import multiprocessing as mp
try:
    mp.set_start_method('spawn', force=True)
except RuntimeError:
    pass  

agent = SAC(env,sac_config,writer)

/home/zakaria/miniconda3/envs/photoen/lib/python3.11/site-packages/torchrl/__init__.py:36: UserWarning: failed to set start method to spawn, and current start method for mp is fork.
  warn(


In [7]:
import time

In [8]:
for i in range(sac_config.total_timesteps):
    episode_count = 0
    agent.start_time = time.time()
    agent.reset_env()
    envs_mean_rewards =[]
    while True:

        episode_count+=1
        agent.global_step+=1
        mean_reward,batch_dones = agent.train()
        envs_mean_rewards.append(mean_reward)

        if (batch_dones==True).all()==True or episode_count==10:
            ens_mean_episodic_return = sum(envs_mean_rewards)
            agent.writer.add_scalar("charts/episodic_return", ens_mean_episodic_return, agent.global_step)
            episode_count=0
            envs_mean_rewards =[]
            agent.reset_env()
            break     

SPS: 67
SPS: 84
SPS: 138
SPS: 146
SPS: 188
SPS: 180
SPS: 260
SPS: 296
SPS: 300
SPS: 253
SPS: 333
SPS: 331
SPS: 425
SPS: 362
SPS: 372


KeyboardInterrupt: 